In [1]:
import numpy as np
from datetime import datetime
from pandas import read_table
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import os
import math

import time

import warnings
warnings.simplefilter('ignore')

In [2]:
import sys
sys.path.insert(0, 'funs')
import period_detect_funs

# functions

## main

In [3]:
def find_dot_in_file_name(file_name):
    i = -1
    while i < 0:
        if file_name[i] == ".":
            return i
        else:
            i = i - 1

In [4]:
def find_last_underline_in_file_name(file_name):
    i = -1
    while i < 0:
        if file_name[i] == "_":
            return i
        else:
            i = i - 1

In [5]:
def parser(s):
    return datetime.strptime(s, '%Y-%m-%d')

In [6]:
def main_period_fun(ts, threshold_low = 3, threshold_range = 3, current_value_flag = True, num_permutations = 100, confidence_interval = 0.99, random_seed = None, plot_flag = False):
    num = len(ts)
    # get acfs
    period_set, raw_acf_set_no = period_detect_funs.get_raw_acfs(ts)
    period_set, mean_acf_set, mean_acf_set_no = period_detect_funs.get_mean_acfs(ts, threshold_low, threshold_range)
    period_set, loess_acf_set, loess_acf_set_no = period_detect_funs.get_loess_acfs(ts, threshold_low, threshold_range, current_value_flag)
    
    # get period with max acf
    loess_max_id = np.argmax(loess_acf_set)
    loess_max_acf = loess_acf_set[loess_max_id]
    loess_period_detect = period_set[loess_max_id]
    
    loess_max_id_no = np.argmax(loess_acf_set_no)
    loess_max_acf_no = loess_acf_set_no[loess_max_id_no]
    loess_period_detect_no = period_set[loess_max_id_no]
    
    mean_max_id = np.argmax(mean_acf_set)
    mean_max_acf = mean_acf_set[mean_max_id]
    mean_period_detect = period_set[mean_max_id]
    
    mean_max_id_no = np.argmax(mean_acf_set_no)
    mean_max_acf_no = mean_acf_set_no[mean_max_id_no]
    mean_period_detect_no = period_set[mean_max_id_no]
    
    raw_max_id_no = np.argmax(raw_acf_set_no)
    raw_max_acf_no = raw_acf_set_no[raw_max_id_no]
    raw_period_detect_no = period_set[raw_max_id_no]
    
    # data = [['period',str(loess_period_detect), str(loess_period_detect_no), str(mean_period_detect), str(mean_period_detect_no), str(raw_period_detect_no)],['acf',loess_max_acf, loess_max_acf_no, mean_max_acf, mean_max_acf_no, raw_max_acf_no]]
    # df = pd.DataFrame(data,columns=['data','loess with weights', 'loess without weights', 'mean with weights', 'mean without weights', 'raw without detrend'])
    dic_pal = {
                        'loess_max_acf':   loess_max_acf,
                        "loess_period_detect":   loess_period_detect,
                        "loess_max_acf_no": loess_max_acf_no,
                        'loess_period_detect_no':   loess_period_detect_no,
                        'mean_max_acf':   mean_max_acf,
                        'mean_period_detect':   mean_period_detect,
                        'mean_max_acf_no':   mean_max_acf_no,
                        'mean_period_detect_no':   mean_period_detect_no,
                        'raw_max_acf_no':   raw_max_acf_no,
                        'raw_period_detect_no':   raw_period_detect_no
                    }
    
    
    # auto period
    # get period hints
    fft_ks = period_detect_funs.getPeriodHints(ts, num_permutations, confidence_interval, random_seed)
    # raw acfs
    raw_periods_no = period_detect_funs.validation_all_hints(fft_ks, num, raw_acf_set_no)
    # mean acfs
    mean_periods_no = period_detect_funs.validation_all_hints(fft_ks, num, mean_acf_set_no)
    mean_periods = period_detect_funs.validation_all_hints(fft_ks, num, mean_acf_set)
    # loess acfs
    loess_periods_no = period_detect_funs.validation_all_hints(fft_ks, num, loess_acf_set_no)
    loess_periods = period_detect_funs.validation_all_hints(fft_ks, num, loess_acf_set)
    
    # data_auto = [['period',str(loess_periods), str(loess_periods_no), str(mean_periods), str(mean_periods_no), str(raw_periods_no)]]
    # df_auto = pd.DataFrame(data_auto,columns=['acf type','loess with weights', 'loess without weights', 'mean with weights', 'mean without weights', 'raw without detrend'])
    dic_auto = {
                        'raw_periods_no':   raw_periods_no,
                        "mean_periods_no":   mean_periods_no,
                        "mean_periods": mean_periods,
                        'loess_periods_no':   loess_periods_no,
                        'loess_periods':   loess_periods
                    }
    
    if plot_flag:
        plt.figure(figsize = (24, 4.5))
        plt.subplot(1,5,1)
        plt.plot(period_set, loess_acf_set)
        plt.grid()
        plt.xlabel('lag')
        plt.ylabel('acf')
        plt.title('loess acf with weights')
        plt.subplot(1,5,2)
        plt.plot(period_set, loess_acf_set_no)
        plt.grid()
        plt.xlabel('lag')
        plt.ylabel('acf')
        plt.title('loess acf without weights')
        plt.subplot(1,5,3)
        plt.plot(period_set, mean_acf_set)
        plt.grid()
        plt.xlabel('lag')
        plt.ylabel('acf')
        plt.title('mean acf with weights')
        plt.subplot(1,5,4)
        plt.plot(period_set, mean_acf_set_no)
        plt.grid()
        plt.xlabel('lag')
        plt.ylabel('acf')
        plt.title('mean acf without weights')
        plt.subplot(1,5,5)
        plt.plot(period_set, raw_acf_set_no)
        plt.grid()
        plt.xlabel('lag')
        plt.ylabel('acf')
        plt.title('raw acf without detrend')
    
    return dic_pal, dic_auto

## period error

In [7]:
def choose_auto_period(auto_periods, real_period):
    # if auto_periods == None:
    #     return 0
    if len(auto_periods) == 0:
        return 0
    abs_diff = np.abs(auto_periods - real_period)
    min_idx = np.argmin(abs_diff)
    return auto_periods[min_idx]
    

In [8]:
def pal_period_fun(raw_period, acf, threshold):
    if acf > threshold:
        return raw_period
    else:
        return 0
    

In [9]:
def error_meauser_num(periods_detect, periods_real, errors):
    assert(len(periods_detect) == len(periods_real))
    abs_diff = np.abs(np.floor(periods_detect) - np.floor(periods_real))
    abs_diff_ratio = abs_diff / np.floor(periods_real)
    num = len(periods_detect)
    errors_sort = np.sort(errors)
    num_error = len(errors_sort)
    error_distr = np.zeros(num_error+1)
    for i in range(num):
        for j in range(num_error):
            if abs_diff_ratio[i] <= errors_sort[j]:
                error_distr[j] = error_distr[j] + 1
                break
        if abs_diff_ratio[i] > errors_sort[-1]:
            error_distr[-1] = error_distr[-1] + 1
    return error_distr

# cran dataset

## csv names and freqs

In [10]:
folder_path = "data/SL_CRAN_TS_data"
file_names = os.listdir(folder_path)
csv_names = np.array([], dtype = object)
freqs = np.array([])
for file_name in file_names:
    if file_name[-4:] == ".csv":
        csv_names = np.append(csv_names, file_name)
        dot_id = find_dot_in_file_name(file_name)
        last_underline_id = find_last_underline_in_file_name(file_name)
        freq = float(file_name[last_underline_id+1:dot_id])
        freqs = np.append(freqs, freq)

## period detection

In [11]:
str_path = "data/SL_CRAN_TS_data/"
num_csv = len(csv_names)
# num_csv = 3
raw_periods_pal = np.full([5, num_csv],np.nan) 
acfs_pal = np.full([5, num_csv],np.nan)# raw_max_acf_no, mean_max_acf_no, mean_max_acf, loess_max_acf_no, loess_max_acf
periods_auto = np.full([5, num_csv],np.nan)
for i in range(num_csv):
    df = pd.read_csv(str_path+csv_names[i])
    data = np.full(len(df),np.nan,dtype = float)
    for j in range(len(df)):
        data[j] = df['value'][j]
    dic_pal, dic_auto = main_period_fun(data, random_seed = 123)
    # print(dic_auto)
    # pal
    raw_periods_pal[0][i] = dic_pal['raw_period_detect_no']
    raw_periods_pal[1][i] = dic_pal['mean_period_detect_no']
    raw_periods_pal[2][i] = dic_pal['mean_period_detect']
    raw_periods_pal[3][i] = dic_pal['loess_period_detect_no']
    raw_periods_pal[4][i] = dic_pal['loess_period_detect']
    acfs_pal[0][i] = dic_pal['raw_max_acf_no']
    acfs_pal[1][i] = dic_pal['mean_max_acf_no']
    acfs_pal[2][i] = dic_pal['mean_max_acf']
    acfs_pal[3][i] = dic_pal['loess_max_acf_no']
    acfs_pal[4][i] = dic_pal['loess_max_acf']
    
    # auto
    periods_auto[0][i] = choose_auto_period(dic_auto['raw_periods_no'], freqs[i])
    periods_auto[1][i] = choose_auto_period(dic_auto['mean_periods_no'], freqs[i])
    periods_auto[2][i] = choose_auto_period(dic_auto['mean_periods'], freqs[i])
    periods_auto[3][i] = choose_auto_period(dic_auto['loess_periods_no'], freqs[i])
    periods_auto[4][i] = choose_auto_period(dic_auto['loess_periods'], freqs[i])


starting computations on 56 cores


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 4956.59it/s]

starting computations on 56 cores



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 4467.60it/s]

starting computations on 56 cores


...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:00<00:00, 6690.07it/s]

starting computations on 56 cores



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:00<00:00, 5630.92it/s]

starting computations on 56 cores



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:00<00:00, 1104.67it/s]


In [12]:
threshold = 0.4
periods_pal = np.full([5, num_csv],np.nan) 
for i in range(len(periods_pal)):
    for j in range(len(periods_pal[0])):
        periods_pal[i][j] = pal_period_fun(raw_periods_pal[i][j], acfs_pal[i][j], threshold)

In [13]:
errors = np.array([0,0.05,0.1,0.15])
num_error = len(errors)
num_acf = len(periods_pal)
error_distr_pal = np.full([num_acf, num_error+1], np.nan)
error_distr_auto = np.full([num_acf, num_error+1], np.nan)
for i in range(num_acf):
    error_distr_pal[i] = error_meauser_num(periods_pal[i], freqs[:num_csv], errors)
    error_distr_auto[i] = error_meauser_num(periods_auto[i], freqs[:num_csv], errors)

In [14]:
# raw_max_acf_no, mean_max_acf_no, mean_max_acf, loess_max_acf_no, loess_max_acf

In [15]:
error_distr_pal

array([[34.,  0.,  0.,  0., 48.],
       [60.,  2.,  1.,  0., 19.],
       [62.,  2.,  1.,  0., 17.],
       [61.,  2.,  1.,  0., 18.],
       [62.,  3.,  0.,  0., 17.]])

In [16]:
error_distr_auto

array([[27.,  3.,  0.,  0., 52.],
       [29.,  2.,  0.,  0., 51.],
       [29.,  2.,  0.,  0., 51.],
       [29.,  2.,  0.,  0., 51.],
       [28.,  3.,  0.,  0., 51.]])